In [119]:
import pandas as pd
import numpy as np 
import json
import ast
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [99]:
df = pd.read_csv('./origen.csv')
test = pd.read_csv('./testear.csv')

# Limpieza de datos
## Variables de tipo numericas
- average rating
- numVotes
- isAdult
- startYear
- endYear
- runtimeMinutes
- seasonNumber
- episodeNumber
- ordering
- language
- isOriginalTitle
- budget
- popularity
- revenue
- runtime

## Variables de tipo str
- titleType
- genres_x
- directors
- writers
- adult
- genres_y
- original_language
- production_company
- production_country
- status
- tagline
- video

## Variables booleanas

Todas las columnas son rotuladas de manera numerica
Las columnas indice, directors, writer, tagline, language, adult fueron eliminadas

In [100]:
df = df.drop('Unnamed: 0', axis=1)

In [101]:
print(df['writers'].str.count(',').max())
# Se pueden tener hasta 1288 escritores
df = df.drop(columns=['writers'])

print(df['directors'].str.count(',').max())
# Se pueden tener hasta 475 directores
# dummiesDirectors = df['directors'].str.get_dummies(sep=',')
# df = pd.concat([df, dummiesDirectors], axis=1)
df = df.drop(columns=['directors'])

1287
475


In [102]:
df = df.drop('tagline', axis = 1)

In [103]:
def columnasDummies(unique_values, column):
    for value in unique_values:
        df[value] = (df[column] == value).astype(int)

In [104]:
columnasDummies(df['adult'].unique(), 'adult')
df = df.drop(columns=['adult'])
df = df.drop(columns=['isAdult'])

In [105]:
columnasDummies(df['titleType'].unique(), 'titleType')
df = df.drop(columns=['titleType'])

In [106]:
dummy_df = pd.get_dummies(df['attributes'], prefix='attributes', prefix_sep='_')
df = pd.concat([df, dummy_df], axis=1)
df = df.drop(columns=['attributes'])

In [107]:
dummy_df = pd.get_dummies(df['original_language'], prefix='original_language', prefix_sep='_')
df = pd.concat([df, dummy_df], axis=1)
df = df.drop(columns=['language'])
df = df.drop(columns=['original_language'])

In [108]:
columnasDummies(df['status'].unique(), 'status')
df = df.drop(columns=['status'])

In [109]:
columnasDummies(df['video'].unique(), 'video')
df = df.drop(columns=['video'])

In [110]:
def extract_and_join_names(row):
    if pd.notna(row):  
        cleaned_json = row.replace("'", '"').strip() 
        try:
            country_list = json.loads(cleaned_json)
            names = [country['name'] for country in country_list]
            return ', '.join(names)
        except json.JSONDecodeError as e:
            return 'Invalid JSON'
    else:
        return ''

In [111]:
dummiesGenero = df['genres_x'].str.get_dummies(sep=',')
df = pd.concat([df, dummiesGenero], axis=1)
df = df.drop(columns=['genres_x'])
df = df.drop(columns=['0'])

In [112]:
df['genre_y'] = df['genres_y'].apply(extract_and_join_names)

In [113]:
df['countries'] = df['production_countries'].apply(extract_and_join_names)

In [114]:
df['companies'] = df['production_companies'].apply(extract_and_join_names)

In [115]:
dummiesGenreY = df['genre_y'].str.get_dummies(sep=',')
df = pd.concat([df, dummiesGenreY], axis=1)
df = df.drop(columns=['genre_y'])
df = df.drop(columns=['genres_y'])

In [116]:
dummiesCountries = df['countries'].str.get_dummies(sep=', ')
df = pd.concat([df, dummiesCountries], axis=1)
df = df.drop(columns=['production_countries'])
df = df.drop(columns=['countries'])

In [117]:
empresas_por_fila = df['companies'].str.split(', ')
todas_las_empresas = [empresa for empresas in empresas_por_fila.dropna() for empresa in empresas]
num_empresas_distintas = len(set(todas_las_empresas))

print("Número de empresas distintas:", num_empresas_distintas)

df = df.drop(columns=['production_companies'])
df = df.drop(columns=['companies'])

Número de empresas distintas: 20311


In [118]:
df_copia = df.copy()

### Modelo baseline

In [120]:
X = df.drop(columns=['averageRating'])
y = df['averageRating']

# 2. Separa el DataFrame 'test' en características (X_test) y la variable objetivo (y_test).
X_test = test.drop(columns=['averageRating'])
y_test = test['averageRating']

# 3. Crea un modelo de línea base (en este caso, la media de y).
baseline_model = LinearRegression()
baseline_model.fit(X, y)

# 6. Realiza predicciones en el conjunto de prueba.
y_pred = baseline_model.predict(X_test)

# 7. Calcula el coeficiente de determinación (R2) para evaluar el rendimiento del modelo en el conjunto de prueba.
r2 = r2_score(y_test, y_pred)
print(f'Coeficiente de determinación (R2): {r2:.2f}')

KeyError: "['averageRating'] not found in axis"